In [178]:

# This cell is to load user and password of my database from an .txt file, which will not be pushed to the repository
# Assign your user name and password to lldb_user and lldb_password variables
with open('db_credentials.txt','r') as file:
    lines = file.readlines()
lldb_user = lines[0]
lldb_password = lines[1]
    

In [179]:
import mysql.connector as connector
from mysql.connector import Error

In [180]:
try:
    connection = connector.connect(user = lldb_user, password = lldb_password, db = 'littlelemondb')
except Error as e:
    print('An error has occurred with an error code: {} \n and message: {}'.format(e.errno,e.msg))

In [181]:
cursor = connection.cursor()
cursor.execute('SHOW tables')

In [182]:
results = cursor.fetchall()
for table in results:
    print(table)

('bookings',)
('customers',)
('deliveries',)
('items',)
('menuitems',)
('menus',)
('orders',)
('staff',)


In [183]:
cursor.execute("""
SELECT CONCAT(bc.FirstName, ' ', bc.LastName) AS 'Customer Full Name', bc.Email, bc.Phone, o.TotalCost
FROM Orders AS o 
LEFT JOIN 
	(SELECT b.BookingID, c.CustomerID, c.FirstName, c.LastName, c.Email, c.Phone
    FROM Bookings AS b
    LEFT JOIN Customers AS c 
    ON b.CustomerID = c.CustomerID) AS bc
ON o.BookingID = bc.BookingID
WHERE o.TotalCost > 60;
""")

print(cursor.column_names)

for record in cursor.fetchall():
    print (record)

('Customer Full Name', 'Email', 'Phone', 'TotalCost')
('Joakim Iversen', 'j.iversen@yahoo.com', 352253781, Decimal('92'))
('Diana Pinto', 'diana.pinto@finance.org.us', 751100242, Decimal('78'))
('Vanessa McCarthy', 'vanessa.m@globalsuperstore.com', 361255597, Decimal('280'))
('Joakim Iversen', 'j.iversen@yahoo.com', 352253781, Decimal('138'))
('Diana Pinto', 'diana.pinto@finance.org.us', 751100242, Decimal('120'))
('Marcos Romero', 'marcos.romero2208@mangatagallo.us', 451252112, Decimal('184'))


In [190]:
def CallStoredProcedure(procedureCallQuery, parameters_tuple = None):
    try:
        cursor = connection.cursor()
        if parameters_tuple == None:
            cursor.execute(procedureCallQuery)
        else:
            cursor.execute(procedureCallQuery, parameters_tuple)
        while True:
            print (cursor.column_names)
            for record in cursor.fetchall():
                print(record)
            if not cursor.nextset():
                break
    except Error as e:
        print('An error has occurred with an error code: {} \n and message: {}'.format(e.errno,e.msg))  
    finally:
        cursor.close()

In [191]:
CallStoredProcedure("CALL GetMaxQuantity();")

('Max Quantity in Order',)
(5,)
()


In [200]:
CallStoredProcedure("CALL ManageBooking(%s,%s);",("2025-02-28",6))

('Booking Status',)
('Table 6 is already booked - booking cancelled',)
()


In [201]:
CallStoredProcedure("CALL AddBooking(%s,%s,%s);",("2022-12-18",99, 99))

An error has occurred with an error code: 1452 
 and message: Cannot add or update a child row: a foreign key constraint fails (`littlelemondb`.`bookings`, CONSTRAINT `customer_id_fk` FOREIGN KEY (`CustomerID`) REFERENCES `customers` (`CustomerID`) ON DELETE CASCADE ON UPDATE CASCADE)


In [ ]:
CallStoredProcedure("CALL UpdateBooking(%s,%s,%s);",("2022-12-10",4, 3))

('Booking Status',)
('Table 4 is already booked - booking cancelled',)
()


In [175]:
# Close the connection
if connection.is_connected():
    cursor.close()
    connection.close()
    print("Connection has been closed")
else:
    print("Connection was closed")

Connection has been closed
